# IoT Kernel

The *IoT Kernel* is a specialized Jupyter kernel that evaluates MicroPython code on connected microcontrollers. Features include:

* discover and connect to wired and wireless microcontrollers (`%discover` and `%connect`)
* synchronize files with the host (`%rsync`)
* softreset (`%softreset`)
* evaluate code on the host (`%%host`)
* exchange variables between microcontrollers and the host (`%store`)
* access the host shell (`%%bash`)
* access other services of *iot49* (`%%service`)

## Magics

Many of these features are accessed via magics. `%lsmagic` lists available magics.

In [1]:
%lsmagic

Line Magic:    -h shows help (e.g. %discover -h)
  %cat         Print contents of named file on microcontroller
  %cd          Change the working directory.
  %connect     Connect to device
  %cp          Copy files between host and microcontroller.
  %discover    Discover available devices
  %gettime     Query microcontroller time
  %loglevel    Set logging level.
  %lsmagic     List all magic functions.
  %mkdirs      Create all directories specified by the path, as needed.
  %name        Name of currently connected microcontroller.
  %pip         Install packages from PyPi
  %platform    sys.platform of currently connected device.
  %rdiff       Show differences between microcontroller and host directories
  %register    Register device
  %rlist       List files on microcontroller
  %rm          Delete files relative to path.
  %rsync       Synchronize microcontroller to host directories
  %softreset   Reset microcontroller. Similar to pressing the reset button.
  %store       Copy 

The `-h` option on any magic returns additional information. E.g.

In [5]:
%discover -h

usage: %discover [-h] [-a] [-v]

Discover available devices

optional arguments:
  -h, --help     show this help message and exit
  -a, --all      list all devices connected to USB ports
  -v, --verbose  show uid


Magics not recognized by the *IoT Kernel* are sent to to underlying ipython kernel for evaluation. E.g.

In [2]:
%%writefile main.py
# sample program
print("Hello from main.py!")

Overwriting main.py


In [3]:
!cat main.py

# sample program
print("Hello from main.py!")


In [4]:
!rm main.py

## MicroPython

At any one time, the kernel is connected to one (or no) microcontroller. Code typed into Jupyter cells is sent to the "connected" microcontroller for evaluation. The `%discover` magic scans USB ports and listens to DNS advertisements for available devices. For each device to which connection is successful, the magic reports the device name, URL, and UID. Device names are specified in [configuration files](#device-configuration)).

**Note:** Use a powered USB hub to reliably connect more than one microcontroller to the Raspberry Pi!

In [7]:
%discover

test-esp32  ws://10.39.40.135:8266  
samd51      serial:///dev/ttyACM0   
stm32       serial:///dev/ttyACM1   
test-esp32  serial:///dev/ttyUSB2   
pico        serial:///dev/ttyUSB0   
nrf52840    serial:///dev/ttyACM2   
huzzah32    serial:///dev/ttyUSB1   


The esp32 is available both over USB (serial:///dev/ttyUSB2) and wirelessly webrepl (ws://10.39.40.135:8266).

In [8]:
%connect "ws://10.39.40.135:8266"

for i in range(5): print(i, i**20)

Connected to test-esp32 @ ws://10.39.40.135:8266
0 0
1 1
2 1048576
3 3486784401
4 1099511627776


Connection to a device is maintained until changed. The `%%connect` cell magic runs code sequentially on one or more processors:

In [8]:
%%connect --all --host

import sys
print("{:18}   ({})".format(sys.platform, sys.implementation.name))


----- HOST

linux                (cpython)

----- test-esp32

esp32                (micropython)

----- samd51

MicroChip SAMD51     (circuitpython)

----- stm32

pyboard              (micropython)

----- test-esp32

esp32                (micropython)

----- pico

esp32                (micropython)

----- nrf52840

nRF52840             (circuitpython)

----- huzzah32

esp32                (micropython)


`%softreset` resets the MicroPython interpreter and has a similar effect to pressing the reset button of the device:

In [8]:
%name

a = 5
print(a)
%softreset
print("this will fail ...")
print(a)

test-esp32
5

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

this will fail ...


Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
NameError: name 'a' isn't defined



Check `%lsmagic` for additional commands.

## Environment Variables

|    Name      |   Default Value          |  Description   |
| ------------ | ------------------------ | :------------- |
| IOT          | `~`                      | |
| IOT49        | `$IOT/iot49`             | |
| IOT_DEVICES  | `$IOT49/devices`         | Location(s) of [device configuration](device_configuration) yaml files, separate with `:` |
| IOT_SECRETS  | `$IOT49/libs/secrets.py` | Location of [*secrets.py*](secrets) |
| IOT_LIBS     | `$IOT49/libs`            | Location(s) of MicroPython libraries, separate with `:` |


## File Management

In additional to the usual file operations (`%cp`, `%rm`, etc), the `%rsync` feature can be used to efficiently copy libraries on the host to the microcontroller.

In [8]:
%connect huzzah32

# list files on microcontroller
%rlist

Connected to huzzah32 @ serial:///dev/ttyUSB1
    139  Dec 31 23:00 1999  boot.py


In [7]:
%cat boot.py

# This file is executed on every boot (including wake-boot from deepsleep)
#import esp
#esp.osdebug(None)
#import webrepl
#webrepl.start()



In [5]:
# show differences but do not commit any operations
%rdiff

Directories match


In [5]:
# synchronize the microcontroller flash to the host
# by default, %rsync deletes files on the microcontroller that are not present on the host
# specify -u to upload data only

%rsync -u

Directories match


(device_configuration)=
## Device Configuration

### Simple

Save device configurations in [yaml format](https://en.wikipedia.org/wiki/YAML) in `~/iot49/devices`. A simple example:

```yaml
huzzah32:
    uid: 24:0a:c4:12:87:7c
    resources:
        - pystone.py
        - secrets.py
        - bno055
```

Resources is a list of python source files and packages stored in path-like environment variable `IOT_LIBS` (defaults to '~/iot49/libs').

In [1]:
%connect huzzah32

# start with a clean slate
%rm -rf /lib

Connected to huzzah32 @ serial:///dev/ttyUSB1


In [1]:
%rsync

ADD     /lib/bno055/bno055.py
ADD     /lib/bno055/bno055_base.py
ADD     /lib/bno055/bno055_test.py
ADD     /lib/pystone.py
ADD     /lib/secrets.py


In [1]:
%rlist

                            lib/
                                bno055/
   6491  Jun 29 18:41 2021          bno055.py
   5598  Jun 29 18:41 2021          bno055_base.py
   1078  Jun 29 18:41 2021          bno055_test.py
   7933  Jun 29 18:41 2021      pystone.py
    317  Jun 29 18:41 2021      secrets.py


In [1]:
%rsync
%softreset

import pystone
pystone.main(2000)

Directories match

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Pystone(1.2) time for 2000 passes = 1.968 seconds
This machine benchmarks at 1016.26 pystones/second


### Custom Destination

By default, resources are installed in the `/lib` folder on the microcontroller. Files like `boot.py` instead go to a special location. Updated configuration:

```yaml
huzzah32:
    uid: 24:0a:c4:12:87:7c
    resources:
        - esp32:
            install-dir: /
            unpack: true
        - pystone.py
        - secrets.py: /
        - bno055
```

`install-dir` overwrites the default (`/lib`) and `unpack` indicates that the resource should not be treated as a package but instead the contents should be uploaded.

`secrets.py: /` is a shorthand for

```yaml
- secrets.py:
    install-dir: /
```

In [1]:
%rsync
%rlist

UPDATE  /boot.py
   1787  Jun 29 18:58 2021  boot.py
                            lib/
                                bno055/
   6491  Jun 29 18:41 2021          bno055.py
   5598  Jun 29 18:41 2021          bno055_base.py
   1078  Jun 29 18:41 2021          bno055_test.py
                                ota32/
     51  Jun 29 18:46 2021          __init__.py
   2048  Jun 29 18:46 2021          open_url.py
   3489  Jun 29 18:46 2021          ota.py
   7933  Jun 29 18:41 2021      pystone.py
    418  Jun 29 18:46 2021  mp_main.py
    317  Jun 29 18:48 2021  secrets.py
     52  Jun 29 18:46 2021  webrepl_cfg.py


In [1]:
%softreset
import boot


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

WebREPL daemon started on ws://10.39.40.163:8266
Started webrepl in normal mode


In [1]:
%connect 'ws://10.39.40.163:8266'

Connected to huzzah32 @ ws://10.39.40.163:8266


In [1]:
import network
wlan = network.WLAN(network.STA_IF)
print(wlan.ifconfig()[0])

10.39.40.163


### Full Example

This particular `stm32` processor has a small on-chip flash partition at `/flash` used for `boot.py` and the code to mount an external flash chip at `/spi`. Libraries that do not need to be on `/flash` are installed in `/spi/lib`.

The configuration also specifies a custom library path that is searched instead of `$IOT_LIBS`.

```yaml
test-stm32:
    uid: 27:00:55:00:09:50:52:42:4e:30:39:20
    libs:
        - $IOT49/libs
        - $IOT/projects/airlift/libs
    install-dir: /spi/lib
    resources:
        - stm32: 
            install-dir: /flash
            unpack: true
        - secrets.py: /spi
        - neopixel.py
        - airlift_client:
            unpack: true
        - mp_server:
            unpack: true
            install-dir: /spi/lib/mp
```

(secrets)=
## secrets.py

A Python file used to define confidential information such as passwords. Example:

```python
# secrets.py

# wifi
wifi_ssid = 'WIFI_SSID'
wifi_pwd  = 'WIFI_PASSWORD'

# timezone
tz_offset = -8*3600    # PST

# mp wireless "repl" password
mp_pwd  = '&(*HFODjpwdlsdf30i%^&'

# webrepl password, 4 .. 9 characters
webrepl_pwd = 'fjlkflsad'

# https://openweathermap.org/
openweathermap_apiid = "354f336ac61afasf2b73af8f"
```

Some of these variables (e.g `webrepl_pwd`) are used by MicroPython and *ide49*.